In [2]:
import requests
import json
import feedparser
import sqlite3
from tqdm import tqdm
from youtube_transcript_api import YouTubeTranscriptApi
import youtube_transcript_api._errors
from ollama import Client

In [3]:
conn=sqlite3.connect("../../db.sqlite3")

In [3]:
feedUrl:str='https://www.youtube.com/feeds/videos.xml?channel_id={channelId}'
transcriptAPI='https://transcriptextractor.com/api/transcripts'

In [4]:
channelList=[a[:-1] for a in open('channelList.csv','rt').readlines()]

In [ ]:
res=requests.get(channelList[0])
channelList[0],res.status_code

In [5]:
channelIdList=[]
for channel in tqdm(channelList):
    res=requests.get(channel)
    if res.status_code !=200:
        print(channel)
        break
    startIndex=res.text.find("externalId")
    channelId=""
    c=0
    while True:
        channelId+=res.text[startIndex+c]
        c+=1
        if res.text[startIndex+c]==",":
            break
    channelIdList.append((channelId.split(":")[1].replace('"',''),channel.split("@")[1]))

100%|██████████| 193/193 [01:32<00:00,  2.09it/s]


In [16]:
conn.execute("""CREATE TABLE IF NOT EXISTS channel (channel_id TEXT , name TEXT)""")
conn.execute("CREATE TABLE IF NOT EXISTS latest_video (channel_id TEXT,video_id TEXT,upload_timestamp TEXT)")
conn.execute("INSERT INTO channel (channel_id,name) VALUES "+",".join([f"('{cid}','{name}')" for cid,name in channelIdList]))
conn.commit()

In [5]:
conn.execute("DELETE FROM latest_video")
conn.execute("DELETE FROM video")
conn.execute("DROP TABLE video")

conn.commit()

In [ ]:
newVideoList=[]
for channelId,channelName in tqdm(channelIdList):
    parsedData=feedparser.parse(feedUrl.format(channelId=channelId))
    for index,entry in (enumerate(parsedData["entries"])):
        
        if latestVideoId:=conn.execute(f"SELECT video_id FROM latest_video WHERE channel_id='{entry["yt_channelid"]}'").fetchone():
            if latestVideoId!=entry["yt_videoid"]:
                conn.execute(f"UPDATE latest_video SET video_id='{entry['yt_videoid']}',upload_timestamp='{entry['published']}' WHERE channel_id='{entry['yt_channelid']}'")
                # print("new video")
                newVideoList.append(entry["yt_videoid"])
            else:
                # print("old video")
                break
        else:
            conn.execute(f"INSERT INTO latest_video (video_id,channel_id,upload_timestamp) VALUES ('{entry['yt_videoid']}','{entry['yt_channelid']}','{entry['published']}')")
            # print("new video entry")
    conn.commit()

In [ ]:
conn.close()

In [ ]:
parsedData['entries']

In [ ]:
newVideoList=[a[0] for a in conn.execute(f"SELECT video_id FROM latest_video").fetchall()]

In [ ]:
videoTranscript={}
for video in tqdm(newVideoList):
    try:
        transcript:youtube_transcript_api._transcripts.Transcript=YouTubeTranscriptApi.list_transcripts(video).find_transcript(['en','hi'])
        if transcript.language_code!='en':
            transcript=transcript.translate("en")
        t=transcript.fetch()
        videoTranscript[video]=" ".join([text['text'] for text in t])
    except Exception as e:
        print(video,e)
        # raise e

In [ ]:
videoTranscript['lMpTRda55Cw']

In [ ]:
client = Client(host='http://localhost:11434')

In [159]:
modelfile='''
FROM llama3:8b-instruct-q6_K
SYSTEM You are a summarizer, summarize every things into points
'''

client.create(model='llama3:extractKnowledge', modelfile=modelfile)


{'status': 'success'}

In [161]:
client.generate('llama3:extractKnowledge',videoTranscript['lMpTRda55Cw'])

{'model': 'llama3:extractKnowledge',
 'created_at': '2024-06-03T17:47:52.100853907Z',
 'response': 'Here are the main points summarized from the video:\n\n* The comedian is releasing deleted jokes as bonus content.\n* He introduces a non-vegetarian joke and asks viewers if they want to hear it.\n* He talks about cricket and how players wear protective gear, including L-Guard (Abdominal Guard) which protects the player\'s abdomen.\n* He shares a humorous anecdote about the early days of cricket, where batsmen didn\'t have helmets and used hats instead.\n* He pokes fun at street cricket in India, saying that players often don\'t have proper equipment or shoes, and sometimes even use Gutka packets as makeshift balls.\n* He talks about gully cricket, a form of informal cricket played on streets, and how it\'s common for only one person to own the bat, making them the captain, selector, and umpire all at once.\n* He shares his experience playing street cricket as a child, where they would i

In [ ]:
type(a)

youtube_transcript_api._transcripts.Transcript